## restructing of create_reflection_rules_test2

1. Restructure RulesDict - consistent description for all rules with some way to note translational rule(s); probably keep Laue sym ops as they are. condrule [cond, rule] where cond is compound condition (e.g. Eq(h, 0) AND Eq(l, 0); rule is defined as rule_expression = 2n (= even).

- next: modify condRulesFromSyops: AND instead of set + get explicit flag for translational rules. - done
- check output with compound conditions - done
- new dict structure; all rules treated consistently - done
- add flag for transrule - how to identify transrule? - done

2. Update all functions that use RulesDict for new structure; remove separateTransRule? - done

next

3. add extra rules for special positions
- conditions are ANDed; rules (where multiple for same condition) are written as separate rules

4. speed up getTruthListfromReflist. Needs huge acceleration, including expanding extra rules from Laue before reflection loop.

5. find group generators and get small set of rules from outset

6. run for all groups - how many have ad-hoc rules?




## Questions 
Are translational rules identical to rules with no conditions?

## Notes
Nothing relating to Wyckoff letter yet

In [1]:
#imports & setup
import cctbx.sgtbx as sg
import sympy as sp

x, y, z = sp.symbols(['x', 'y', 'z'])
h, k, l = sp.symbols(['h', 'k', 'l'])

In [2]:
#config
sgNum = 142 #ok
#sgNum = 37  #ok
#sgNum = 1 # error - no trans rule (primative) 
#sgNum = 230 #ok but some redundancies
#sgNum = 161 # ok
#sgNum = 194 # error - no trans rule (primative) 

In [3]:
def eliminate_xyz(ff):

    u = {'h': sp.symbols('u_h'), 'k': sp.symbols('u_k'), 'l': sp.symbols('u_l')}

    def find_xyz(expression):
        if expression.coeff(x) != 0:
            return x
        if expression.coeff(y) != 0:
            return y
        if expression.coeff(z) != 0:
            return z

    for hkl in ['h', 'k', 'l']:
        expr =  ff.coeff(hkl)
        xyz_var = find_xyz(expr)

        if xyz_var != None:
            ff = ff.subs(xyz_var, sp.solve(expr - u[hkl], xyz_var)[0])

        conds = []
        for hkl in ['h', 'k', 'l']:
            coef = ff.expand().coeff(u[hkl])
            if coef != 0:
                conds += [sp.Eq(ff.expand().coeff(u[hkl]), 0)]

        #rule: this expression is odd
        rule = ff.subs({u['h']:0, u['k']:0, u['l']:0})
            
    return (ff, set(conds), rule) # use set to remove duplicates
    



# new version that adds conditions (rather than set) and includes third set element that is True for translational rule
def condRulesFromSymOps(symxyz):

    cond_rule_list = []
    
    for sym in symxyz:
        xp, yp, zp = sp.simplify(sym)
        ff = 2 * h * (xp - x) + 2 * k * (yp - y) + 2 * l * (zp - z)
        ff, conds, rule = eliminate_xyz(ff)
        #display('sym:', sym, 'ff:', ff, 'conds:', conds, 'rule:', rule)
        #print('\n')

        #change conds set to sp.And of condition
        #print('conds', conds)
        if conds == set():
            compound_conds = sp.true
        else:
            conds_list = list(conds)
            #print('conds_list', conds_list)
            compound_conds = conds_list.pop(0) # first condition
            for cond in conds:            # all other conditions
                compound_conds = sp.And(compound_conds, cond)
        
        if rule != 0:
            if [[conds, rule]] not in cond_rule_list:
                #cond_rule_list += [[conds, rule]]
                
                if (xp.coeff(x) == 1 and xp.coeff(y) == 0 and xp.coeff(z) == 0 
                    and yp.coeff(x) == 0 and yp.coeff(y) == 1 and yp.coeff(z) == 0 
                    and zp.coeff(x) == 0 and zp.coeff(y) == 0 and zp.coeff(z) == 1):
                
                    _trans = True # purely translational rule
                else:
                    _trans = False
                    
                cond_rule_list += [[compound_conds, rule, _trans]]
                print('??? cond, rule, sym, ff:', [conds, rule], sym, ff) # delete print ###########
        #display('cond_rule_list, compound_conds, rule:', conds, compound_conds, rule)
    return cond_rule_list

def symxyzFromSpacegroup(sgNum):
    sgi = sg.space_group_info(sgNum)
    spaceGroup = sgi.group()
    #print(sgi.symbol_and_number())
    symxyz = [str(s.as_xyz()) for s in spaceGroup.all_ops()]
    return symxyz

def separateTransRule(cond_rule_list):
    # return None for transRule if no centring rules
    transRule, other_rules = None, []
    for _cond_rule in cond_rule_list:
        if _cond_rule[0] == set():       # pick out translational rule (last one - might be bettwe with first one?)
            transRule = _cond_rule[1]
        else:
            other_rules += [_cond_rule]  # if not trans rule then add to other rule list
    return (transRule, other_rules)


def simplify_rule(rule):
    # updated version of previous simplify_rule (which had a bug). This one works for single miller index
    # not applied if more than one index - might want to create a better algorithm.
    h, k, l = sp.symbols(['h', 'k', 'l'])
    
    new_rule = rule.subs(((h, 0), (k, 0), (l, 0))) # get constant part

    for hkl in [h, k, l]:
        c = rule.coeff(hkl)
        if c != 0:  #skip if no coefficient for this miller index (h, k or l)

            (num, denom) = sp.simplify(sp.fraction(2/c))

            cnew = sp.Abs(sp.Rational(2, num)) # positive value is equivalent
            if sp.Abs(cnew) > 1: # rule is trivial - set to zero
                cnew = 0
            
            new_rule += hkl * cnew
        
    #print('Old rule: ', rule, 'New rule: ', new_rule) ########### remove after testing
    
    # don't simplify if more than one non-zero coefficient (haven't figured out how to yet!)
        
    #print(rule.coeff(h), rule.coeff(k), rule.coeff(l))
    #print(int(rule.coeff(h) != 0), int(rule.coeff(k) != 0), int(rule.coeff(l) != 0))
    
    if int(rule.coeff(h) != 0) + int(rule.coeff(k) != 0) + int(rule.coeff(l) != 0) > 1:
        return sp.simplify(rule)
    else:
        return sp.simplify(new_rule)


# def ss(n, d):
#     if d == 1:
#         n1 = n % 2
#     else:
#         #n1 = n % (2 * d) # always +ve
#         n1 = (n + 2 * d) % (2 * d) - (2 * d) # between -1 and +1 (but -1 for 1,1)
#     return n1



def simplify_rules(cond_rule_list):
    for cond_rule in cond_rule_list:
        cond_rule[1] = simplify_rule(cond_rule[1])
    return cond_rule_list

def remove_trivial_rules(cond_rule_list):
    
    new_cond_rule_list = []
    
    for cond_rule in cond_rule_list:
        #print('cond_rule:', cond_rule)
        for hkl in ['h', 'k', 'l']:
            if cond_rule[1].coeff(hkl) % 2 == 0:
                #print('before:', cond_rule[1])
                cond_rule[1] = cond_rule[1].subs({hkl: 0})
                #print('after:', cond_rule[1])
        if cond_rule[1] != 0:
            new_cond_rule_list += [cond_rule]

    return new_cond_rule_list

def remove_duplicate_rules(cond_rule_list):
    new_cond_rule_list = []
    
    for cond_rule in cond_rule_list:
        _new = True
        for new_cond_rule in new_cond_rule_list:
            if cond_rule == new_cond_rule:
                _new = False
                break
        if _new:
            new_cond_rule_list += [cond_rule]      
            
    return new_cond_rule_list

def apply_laueSym_to_cond_rule(cond_rule, laueSym):
    
    condset, rule = cond_rule
    subs = {'h': laueSym[0], 'k': laueSym[1], 'l': laueSym[2]}
    
    new_condset = set()
    
    for cond in condset:
        new_condset.add(sp.simplify(cond.subs(subs, simultaneous=True)))
    
    rule = sp.simplify(simplify_rule(rule.subs(subs, simultaneous=True)))
    return [new_condset, rule]
    
    

def remove_laue_duplicates(cond_rule_list, laueSymList):
    # might be better to re-order list first?
    new_cond_rule_list = []
    
    for cond_rule in cond_rule_list:
        _new = True
        for laue in laueSymList:
            
            new_cond_rule = apply_laueSym_to_cond_rule(cond_rule, laue)
            
            for _cr in new_cond_rule_list:
                if _cr == new_cond_rule:
                    _new = False
                    break
        if _new:
            new_cond_rule_list += [cond_rule]
      
    return new_cond_rule_list   


def getTruthListfromReflist(hklList, rulesDict):
    #display('rulesDict:', rulesDict)
    h, k, l = sp.symbols(['h', 'k', 'l'])
    truthListLength = 1 + len(rulesDict['nonTransCondRules'])
    truthDict = dict()
    
    for hkl in hklList:

        truthList = [True] * truthListLength
            
        for laue in laueSymList: # might not be necessary to apply laue to transrule
            dummy, transRule =  apply_laueSym_to_cond_rule([{}, rulesDict['transRule']], laue)
            if sp.N(transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]})) % 2 != 0: # expression not even; rule broken
                truthList[0] = False
                break

        for i in range(len(rulesDict['nonTransCondRules'])):
            for laue in laueSymList:
                #conds, rule =  apply_laueSym_to_cond_rule(nonTransCondRules[i], laue)
                conds, rule =  apply_laueSym_to_cond_rule(rulesDict['nonTransCondRules'][i], laue)
                #display("rulesDict['nonTransCondRules'][i], conds (laue), rule(laue)", rulesDict['nonTransCondRules'][i], conds, rule)
                for cond in conds:                       
                    condhkl = cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}) # evaluation for hkl
                    #print(hkl, cond, condhkl)

                    if not condhkl:   # skip if any condition violated
                        break        # break out of conds loop as soon as one fails                                                
                                                       
                #print(i, conds, cond, rule, condhkl)
                if condhkl: #condition is True
                    if sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0: # expression not even; rule broken
                        truthList[i+1] = False
                        break   # break out of laue loop as soon as a rule fails
                #print(list(map(int, truthList)))
            #print('R%i' % (i+2), truthList[i+1])
        truthDict[tuple(hkl)] = truthList
        #print(truthList)
    return truthDict


def isAllowedByTensorCalc(hklList, sg_no, wyckoff_letter = None):
    import TensorScatteringClass as ten
    
    t=ten.TensorScatteringClass(spacegroup_number = sg_no, wyckoff_letter = 'a', verbose = False)
    if wyckoff_letter == None:
        wyckoff_letter = t.all_letters[0]
        print(t.all_letters, wyckoff_letter)
      
        hklAllowed = []
        
        for hkl in hklList:
            t.SF_symmetry(t.sitevec, hkl, t.sglist)
            hklAllowed += [t.gen_scalar_allowed == 1]
    
    return hklAllowed
    
def genReflist(hklMax):
    refList = []
    for h in range(-hklMax, hklMax+1):
        for k in range(-hklMax, hklMax+1):
            for l in range(-hklMax, hklMax+1):
                 refList += [[h, k, l]]
    return refList
                    
def laueClassEquivRefs(hkl, laueSymList):       
    h, k, l = hkl
    hklNewList = []
    for laue in laueSymList:

            newhkl = [eval(laue[0]), eval(laue[1]), eval(laue[2])]
            hklNewList += [newhkl]
    return hklNewList
       

#### test this    
def laueSymListFromSpacegroup(sgNum):
    from sympy.parsing.sympy_parser import parse_expr as p
    from sympy.matrices import Matrix
    
    spaceGroup = sg.space_group_info(sgNum).group()
    
    ashklList = [] 
    for op in spaceGroup.build_derived_laue_group().all_ops():
        asxyz = op.as_xyz().split(',')
   
        m = sp.Matrix([
            [p(asxyz[0]).coeff('x'), p(asxyz[0]).coeff('y'), p(asxyz[0]).coeff('z')],
            [p(asxyz[1]).coeff('x'), p(asxyz[1]).coeff('y'), p(asxyz[1]).coeff('z')],
            [p(asxyz[2]).coeff('x'), p(asxyz[2]).coeff('y'), p(asxyz[2]).coeff('z')]])

        #minv = m.inv() # doesn't work - try transpose - looks ok? but transpose without inv may be ok??
        minv = m.inv().T
        
        #v = sp.simplify(minv * sp.Matrix(['x', 'y', 'z']))
        vhkl = sp.simplify(minv * sp.Matrix(['h', 'k', 'l']))
        
        
        ashklList += [[str(vhkl[0]),str(vhkl[1]),str(vhkl[2])]]

    return(ashklList)
        

def combos(N):
    # return list of all combinations of rules with rule 1 present in all cases
    from itertools import combinations

    _ll, _lc = [], [[1]]
    for n in range(1, N):
        _ll += list(combinations(range(2, N+1), n))

    for _comb in _ll:
        _lc += [[1] + list(_comb)]
        
    return _lc




In [4]:
sg_info = sg.space_group_info(sgNum).symbol_and_number()
symxyz = symxyzFromSpacegroup(sgNum)
condRulesAll = condRulesFromSymOps(symxyz)
laueSymList = laueSymListFromSpacegroup(sgNum)

condRulesAllReduced = remove_duplicate_rules(simplify_rules(remove_trivial_rules(condRulesAll)))


# CondRulesTransList: condition (AND form; True if no condition), rule (expression that must be even), True/False for translational rule
rulesDict = {'sg_info': sg_info, 'laueSymList': laueSymList, 'CondRulesTransList': condRulesAllReduced}    

display(rulesDict)


??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, l/2] -y+1/4,x+3/4,z+1/4 h*u_h + k*u_k + l/2
??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, 3*l/2] y+1/4,-x+1/4,z+3/4 h*u_h + k*u_k + 3*l/2
??? cond, rule, sym, ff: [{Eq(l, 0), Eq(h - k, 0)}, 2*k] y+1/4,x+3/4,-z+3/4 h*u_h + 2*k*(1 - u_h/2) + l*u_l
??? cond, rule, sym, ff: [{Eq(h, 0)}, -l] -x,y,z-1/2 h*u_h - l
??? cond, rule, sym, ff: [{Eq(k, 0)}, -h] x-1/2,-y,z -h + k*u_k
??? cond, rule, sym, ff: [{Eq(l, 0)}, -k] x,y-1/2,-z -k + l*u_l
??? cond, rule, sym, ff: [{Eq(h + k, 0)}, -k - 3*l/2] -y-1/4,-x-3/4,z-3/4 h*u_h + 2*k*(u_h/2 - 1/2) - 3*l/2
??? cond, rule, sym, ff: [{Eq(h - k, 0)}, -k - l/2] y-1/4,x-1/4,z-1/4 h*u_h + 2*k*(-u_h/2 - 1/2) - l/2
??? cond, rule, sym, ff: [set(), h + k + l] x+1/2,y+1/2,z+1/2 h + k + l
??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, 3*l/2] -y+3/4,x+5/4,z+3/4 h*u_h + k*u_k + 3*l/2
??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, 5*l/2] y+3/4,-x+3/4,z+5/4 h*u_h + k*u_k + 5*l/2
??? cond, rule, sym, ff: [{Eq(l, 0), E

{'sg_info': 'I 41/a c d :2 (No. 142)',
 'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'CondRulesTransList': [[Eq(h, 0) & Eq(k, 0), l/2, False],
  [Eq(h, 0), l, False],
  [Eq(k, 0), h, False],
  [Eq(l, 0), k, False],
  [Eq(h + k, 0), -k - 3*l/2, False],
  [Eq(h - k, 0), -k - l/2, False],
  [True, h + k + l, True],
  [Eq(k, 0) & Eq(l, 0), h, False],
  [Eq(h, 0) & Eq(l, 0), k, False],
  [Eq(h, 0) & Eq(k, 0), l, False],
  [Eq(h, 0), k, False],
  [Eq(k, 0), l, False],
  [Eq(l, 0), h, False],
  [Eq(h + k, 0), -k - l/2, False],
  [Eq(h - k, 0), k + l/2, False]]}

In [5]:
condRulesAll

[[Eq(h, 0) & Eq(k, 0), l/2, False],
 [Eq(h, 0) & Eq(k, 0), l/2, False],
 [Eq(l, 0) & Eq(h - k, 0), 0, False],
 [Eq(h, 0), l, False],
 [Eq(k, 0), h, False],
 [Eq(l, 0), k, False],
 [Eq(h + k, 0), -k - 3*l/2, False],
 [Eq(h - k, 0), -k - l/2, False],
 [True, h + k + l, True],
 [Eq(h, 0) & Eq(k, 0), l/2, False],
 [Eq(h, 0) & Eq(k, 0), l/2, False],
 [Eq(k, 0) & Eq(l, 0), h, False],
 [Eq(h, 0) & Eq(l, 0), k, False],
 [Eq(h, 0) & Eq(k, 0), l, False],
 [Eq(l, 0) & Eq(h - k, 0), 0, False],
 [Eq(h, 0), k, False],
 [Eq(k, 0), l, False],
 [Eq(l, 0), h, False],
 [Eq(h + k, 0), -k - l/2, False],
 [Eq(h - k, 0), k + l/2, False]]

In [6]:
crashBash

NameError: name 'crashBash' is not defined

In [88]:
def condRulesFromSymOpsWykoff(sgNum, wyk):
    
    cond_rule_list = []

    sgi = sg.space_group_info(sgNum)
    w = sgi.wyckoff_table()
    spaceGroup = sgi.group()
    symxyz = [str(s.as_xyz()) for s in spaceGroup.all_ops()]  # get all sym ops as xyz

    wykxyz = str(w.position(wyk).special_op())   # get  Wykoff position for Wykoff symbol wyk
    #print('wykxyz:', wykxyz)
    X, Y, Z = sp.sympify(wykxyz.split(','))       # sympy expressions for X, Y, Z components of Wykoff
    
    
    for sym in symxyz:

        v = sp.Matrix(sp.sympify(sym.split(',')))
        
        # matrix part of sym op
        M = sp.Matrix([
            [v[0].coeff(x), v[0].coeff(y), v[0].coeff(z)],
            [v[1].coeff(x), v[1].coeff(y), v[1].coeff(z)],
            [v[2].coeff(x), v[2].coeff(y), v[2].coeff(z)]
        ])

        # vector part of sym op
        T = v.subs({x: 0, y: 0, z: 0})
        
        axx, axy, axz, ayx, ayy, ayz, azx, azy, azz = list(M)
        ax0, ay0, az0 = list(T)
        
        C1 = h * (axx-1) + k * ayx + l * azx
        C2 = h * axy + k * (ayy-1) + l * azy
        C3 = h * axz + k * ayz + l * (azz-1)
        RR = h * ax0 + k * ay0 + l * az0
        
        chi = X * C1 + Y * C2 + Z * C3 + RR
        
        chix, chiy, chiz, chi1 = chi.coeff(x), chi.coeff(y), chi.coeff(z), chi.subs({x: 0, y: 0, z: 0})
        #print('chix, chiy, chiz', chix, ',', chiy, ',', chiz)
        
        C = sp.simplify(sp.simplify(sp.Eq(chix, 0)) & sp.simplify(sp.Eq(chiy, 0)) & sp.simplify(sp.Eq(chiz, 0))) #compound condition
        R = chi1
        #display(C, R)

        # simplify rule according to conditions (e.g.h=0)
        dummy = sp.sympify('dummy')
        and_expression = C & dummy # and with dummy to make sure args are always with AND
        Rs = R.subs(())   
        for and_arg in and_expression.args:
            if and_arg != dummy:
                #print(and_arg.args)
                Rs = Rs.subs(*and_arg.args)

        #print('~~~', R, '~~~', Rs, '~~~', [C, Rs])
        
        if Rs != 0:
            cond_rule_list += [[C, Rs]]
        #print("=================next sym===========================")
    return cond_rule_list

In [91]:
sgNum = 142


sg_info = sg.space_group_info(sgNum).symbol_and_number()
symxyz = symxyzFromSpacegroup(sgNum)
condRulesAll = condRulesFromSymOps(symxyz)
print('\n\n\nold code - genearl pos only')
display(len(condRulesAll), condRulesAll)
#new
sgi = sg.space_group_info(sgNum)
w = sgi.wyckoff_table()

for wyk in [w.position(i).letter() for i in range(w.size())]:
    wykxyz = str(w.position(wyk).special_op())
    print(wyk,':\t', wykxyz)
    cr = condRulesFromSymOpsWykoff(sgNum, wyk)
    display(len(cr), cr)
    print('\n\n\n')




??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, l/2] -y+1/4,x+3/4,z+1/4 h*u_h + k*u_k + l/2
??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, 3*l/2] y+1/4,-x+1/4,z+3/4 h*u_h + k*u_k + 3*l/2
??? cond, rule, sym, ff: [{Eq(l, 0), Eq(h - k, 0)}, 2*k] y+1/4,x+3/4,-z+3/4 h*u_h + 2*k*(1 - u_h/2) + l*u_l
??? cond, rule, sym, ff: [{Eq(h, 0)}, -l] -x,y,z-1/2 h*u_h - l
??? cond, rule, sym, ff: [{Eq(k, 0)}, -h] x-1/2,-y,z -h + k*u_k
??? cond, rule, sym, ff: [{Eq(l, 0)}, -k] x,y-1/2,-z -k + l*u_l
??? cond, rule, sym, ff: [{Eq(h + k, 0)}, -k - 3*l/2] -y-1/4,-x-3/4,z-3/4 h*u_h + 2*k*(u_h/2 - 1/2) - 3*l/2
??? cond, rule, sym, ff: [{Eq(h - k, 0)}, -k - l/2] y-1/4,x-1/4,z-1/4 h*u_h + 2*k*(-u_h/2 - 1/2) - l/2
??? cond, rule, sym, ff: [set(), h + k + l] x+1/2,y+1/2,z+1/2 h + k + l
??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, 3*l/2] -y+3/4,x+5/4,z+3/4 h*u_h + k*u_k + 3*l/2
??? cond, rule, sym, ff: [{Eq(k, 0), Eq(h, 0)}, 5*l/2] y+3/4,-x+3/4,z+5/4 h*u_h + k*u_k + 5*l/2
??? cond, rule, sym, ff: [{Eq(l, 0), E

20

[[Eq(h, 0) & Eq(k, 0), l/2, False],
 [Eq(h, 0) & Eq(k, 0), 3*l/2, False],
 [Eq(l, 0) & Eq(h - k, 0), 2*k, False],
 [Eq(h, 0), -l, False],
 [Eq(k, 0), -h, False],
 [Eq(l, 0), -k, False],
 [Eq(h + k, 0), -k - 3*l/2, False],
 [Eq(h - k, 0), -k - l/2, False],
 [True, h + k + l, True],
 [Eq(h, 0) & Eq(k, 0), 3*l/2, False],
 [Eq(h, 0) & Eq(k, 0), 5*l/2, False],
 [Eq(k, 0) & Eq(l, 0), h, False],
 [Eq(h, 0) & Eq(l, 0), k, False],
 [Eq(h, 0) & Eq(k, 0), l, False],
 [Eq(l, 0) & Eq(h - k, 0), 4*k, False],
 [Eq(h, 0), k, False],
 [Eq(k, 0), l, False],
 [Eq(l, 0), h, False],
 [Eq(h + k, 0), -k - l/2, False],
 [Eq(h - k, 0), k + l/2, False]]

g :	 x,y,z


20

...

29

[[True, k/2 + l/4],
 [True, h/2 + 3*l/4],
 [True, -k/2 + l/4],
 [True, h/2 - l/4],
 [True, h/2 + k/2 + l/2],
 [True, -k/2 - l/4],
 [True, -k - l/2],
 [True, -h/2 - k/2 - l],
 [True, -l/2],
 [True, -h/2 - k/2],
 [True, -k/2 - l/4],
 [True, -h/2 - k - 3*l/4],
 [True, -k/2 - l/4],
 [True, h/2 + k/2 + l/2],
 [True, h/2 + k + 3*l/4],
 [True, h + k/2 + 5*l/4],
 [True, h/2 + 3*l/4],
 [True, h + k/2 + l/4],
 [True, h/2 + k/2 + l/2],
 [True, h + k + l],
 [True, h/2 + k/2 + l/2],
 [True, h/2 + l/4],
 [True, h/2 - k/2],
 [True, -l/2],
 [True, h/2 + k/2],
 [True, l/2],
 [True, h/2 + l/4],
 [True, -k/2 - l/4],
 [True, h/2 + l/4]]





a :	 0,1/4,3/8


30

[[True, k/2 + l/4],
 [True, h/2 + 3*l/4],
 [True, -k/2 - l/4],
 [True, h/2 - 3*l/4],
 [True, h/2 + k/2],
 [True, -l/2],
 [True, -k/2 - 3*l/4],
 [True, -k - l],
 [True, -h/2 - k/2 - 3*l/2],
 [True, -l/2],
 [True, -h/2 - k/2],
 [True, -k/2 - 3*l/4],
 [True, -h/2 - k - 3*l/4],
 [True, -k/2 - l/4],
 [True, h/2 + k/2 + l/2],
 [True, h/2 + k + 3*l/4],
 [True, h + k/2 + 5*l/4],
 [True, h/2 + l/4],
 [True, h + k/2 - l/4],
 [True, h/2 + k/2 + l/2],
 [True, h + k + l/2],
 [True, h/2 + k/2],
 [True, h/2 - l/4],
 [True, h/2 - k/2 - l/2],
 [True, -l],
 [True, h/2 + k/2],
 [True, l/2],
 [True, h/2 - l/4],
 [True, -k/2 - l/4],
 [True, h/2 + l/4]]

In [ ]:
## get rules for group
## get reflist for tensor class
## .... calculate rules


import pickle

filename = '775af26c-38bd-4138-8e4b-867a1dcda47a' # full model hkl to 5
with open(filename,"rb") as f0:
    (reflist, out) = pickle.load(f0)



In [ ]:
# for each sg/site, create a disctionary of classes of tensors    
for _sg in out:
    if _sg[1] != {}:
        #print('#%i' % sg[0])
        for site in _sg[1].keys():
            #print(site)
            #print(_sg[1][site])
            
            _sg[1][site]['classes'] = {}
            
            for ref in _sg[1][site]['ref'].keys():
                #print(ref)
                ranks = _sg[1][site]['ref'][ref]['ranks']
                #print(ranks)
    
                # is ref in existing class or need a new class?
                new_class = True
                for cls in _sg[1][site]['classes']:
                    #if ranks in _sg[1][site]['classes'][cls]['ranks']:
                    if ranks == _sg[1][site]['classes'][cls]['ranks']:
                        _sg[1][site]['classes'][cls]['refs'] += [ref]
                        new_class  = False
                    #else:
                    #    break
                
                if new_class:
                    if not bool(_sg[1][site]['classes']): # empty dict
                        class_no = 1
                    else:
                        class_no = max(_sg[1][site]['classes'].keys()) + 1 
                        
                    _sg[1][site]['classes'][class_no] = {'refs': [ref], 'ranks': ranks}
                    
                        


In [ ]:
# get truth list for one spacegroup and save (time-consuming - load from pickle for test)


#filename = "tt_tmp_01aug" # this version contains the two ad-hoc rules
#filename = "tt_tmp_no_adhoc_2" # this version does not contain the two ad-hoc rules (l/2 = even and l/2+2 = even)
#filename = "tt_tmp_no_adhoc_3" # one ad-hoc rule: l/100 = even (i.e. l = 0)
filename = "tt_tmp_no_adhoc_4" # single ad-hoc rule: l/2 = even
import pickle

# comment out to save new data
#tt = getTruthListfromReflist(reflist, rulesDict)
#with open(filename,"wb") as f0:
#    pickle.dump(tt, f0)

# uncomment to use saved rules
with open(filename,"rb") as f0:
    tt = pickle.load(f0)

In [ ]:
### get rules after removing duplicates from complete set; start with small sets and increase till one is not inconsistent

from sympy.logic import SOPform, POSform
from sympy import symbols

#all refs for #161 site a class 1
#allowed_list = out[sgNum-1][1]['a']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['a']['classes'][2]['refs'] #ok but full rules give more complex (possibly euivalent) solution so small set is best
#allowed_list = out[sgNum-1][1]['d']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['e']['classes'][5]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help

# look at c1 + c2 (don't work individually) # OK:  𝑟1∧(¬𝑟4∨¬𝑟6∨¬𝑟7)
#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs'] + out[sgNum-1][1]['c']['classes'][2]['refs']

# look at e1 + e5 (don't work individually) # OK: 𝑟1∧𝑟2∧¬𝑟7
#allowed_list = out[sgNum-1][1]['e']['classes'][1]['refs'] + out[sgNum-1][1]['e']['classes'][5]['refs']

allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs']



#reflist

def allowed(hkl, allowed_list):
    return tuple(hkl) in allowed_list

allRules = [rulesDict['transRule']] + rulesDict['nonTransCondRules']

import numpy as np

truthListList = []
for ref in tt.keys():
    #print(ref, list(map(int,tt[ref])))
    truthListList += [list(map(int,tt[ref]))]
truthListListByRuleBig = np.array(truthListList).T.tolist() # includes duplicates



#remove duplictaes rules/truths
truthListListByRuleSmall, allRulesSmall = [], []
#truthListListByRuleSmall = []

for ruleNum in range(len(truthListListByRuleBig)):
    if truthListListByRuleBig[ruleNum] in truthListListByRuleSmall:
        print('Rule %i duplicated' % ruleNum)
    else:
        truthListListByRuleSmall += [truthListListByRuleBig[ruleNum]]
        allRulesSmall += [allRules[ruleNum]]

print('Rules used:')
for i in range(1, len(allRulesSmall)):
    print('r%i:\t' % i, allRulesSmall[i])
        
len(truthListListByRuleSmall)
### down to 9 rules...

# mofify getMinterms funtion
def getMinterms1(hkl_list, ruleTruths):
    
    N = len(ruleTruths)
    minterms, dontcares = [], []
    
    for i in range(2 ** N):
        truth = [bool(int(s)) for s in list(format(i, '0%sb' % N))]
        rules_allowed = None
        for ii in range(len(hkl_list)): ########## use index and not hkl
            if truth == [truthList[ii] for truthList in ruleTruths]:  #truth pattern matches pattern in truth table
                #if allowed(hkl_list[ii]) == rules_allowed or rules_allowed == None:
                if allowed(hkl_list[ii], allowed_list) == rules_allowed or rules_allowed == None:
                    #rules_allowed = allowed(hkl_list[ii])
                    rules_allowed = allowed(hkl_list[ii], allowed_list)
                    
                else:
                    raise Exception('inconsistent rules:', truth, ' same set of truths have reflections that are allowed and forbidden so no combination of the rules can work') # 
    
        if rules_allowed == None:
            dontcares += [truth]
        elif rules_allowed == True:
            minterms += [truth]
      
    return (minterms, dontcares)

N = len(truthListListByRuleSmall)
for rules_combination in combos(N):
    rules_comb_0 = [i-1 for i in rules_combination] # subtract 1 so rule 1 is list item 0
    smallrules = [truthListListByRuleSmall[i] for i in rules_comb_0]
    #getMinterms1(reflist, smallrules)
    #print(rules_combination)
    try:
        (minterms, dontcares) = getMinterms1(reflist, smallrules)
        symList = symbols(['r%i' % i for i in rules_combination]) # create a list of symbols, one per rule
        display(POSform(symList, minterms, dontcares))
        print('Found consistent combination of rules', rules_combination)
        print('====================', len(smallrules), len(minterms), len(dontcares))
        break # stop after the first consistent compound rule (remove break to compute all)
    except:
        pass
        #print('rules inconsistent')




# from sympy.logic import SOPform
# from sympy import symbols
# N = len(truthListListByRuleSmall)
# symList = symbols(['r%i' % i for i in range(1, N+1)]) # create a list of symbols, one per rule

# #(minterms, dontcares) =  getMinterms1(hkl_list, truthListListByRuleSmall)
# (minterms, dontcares) =  getMinterms1(reflist, truthListListByRuleSmall)
# #SOPform([r1, r2, r3], minterms, dontcares)
# print(minterms)
# print(dontcares)
# display(SOPform(symList, minterms, dontcares))
# display(POSform(symList, minterms, dontcares))

In [ ]:
#### tmp to delete
display(allRules)
display(allRulesSmall)

In [ ]:
### function form of previous cell - experimental - delete if doesn't work
###############################################################################



#all refs for #161 site a class 1
#allowed_list = out[sgNum-1][1]['a']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['a']['classes'][2]['refs'] #ok but full rules give more complex (possibly euivalent) solution so small set is best
#allowed_list = out[sgNum-1][1]['d']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['e']['classes'][5]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help

# look at c1 + c2 (don't work individually) # OK:  𝑟1∧(¬𝑟4∨¬𝑟6∨¬𝑟7)
#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs'] + out[sgNum-1][1]['c']['classes'][2]['refs']

# look at e1 + e5 (don't work individually) # OK: 𝑟1∧𝑟2∧¬𝑟7
#allowed_list = out[sgNum-1][1]['e']['classes'][1]['refs'] + out[sgNum-1][1]['e']['classes'][5]['refs']

#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs']



#reflist

def findCompoundRule(allowed_list):
    #pass more valiables (tt, reflist)

    from sympy.logic import SOPform, POSform
    from sympy import symbols

    def allowed(hkl, allowed_list):
        return tuple(hkl) in allowed_list

    allRules = [rulesDict['transRule']] + rulesDict['nonTransCondRules']

    import numpy as np

    truthListList = []
    for ref in tt.keys():
        #print(ref, list(map(int,tt[ref])))
        truthListList += [list(map(int,tt[ref]))]
    truthListListByRuleBig = np.array(truthListList).T.tolist() # includes duplicates



    #remove duplictaes rules/truths
    truthListListByRuleSmall, allRulesSmall = [], []
    #truthListListByRuleSmall = []

    for ruleNum in range(len(truthListListByRuleBig)):
        if truthListListByRuleBig[ruleNum] in truthListListByRuleSmall:
            print('Rule %i duplicated' % ruleNum)
        else:
            truthListListByRuleSmall += [truthListListByRuleBig[ruleNum]]
            allRulesSmall += [allRules[ruleNum]]

    print('Rules used:')
#    for i in range(1, len(allRulesSmall)):
#        print('r%i:\t' % i, allRulesSmall[i])
    for i in range(len(allRulesSmall)):
        print('r%i:\t' % (i + 1), allRulesSmall[i]) # modified to fix problem due to indexing from 0 while labelleing from 1

    len(truthListListByRuleSmall)
    ### down to 9 rules...

    # mofify getMinterms funtion
    def getMinterms1(hkl_list, ruleTruths):

        N = len(ruleTruths)
        minterms, dontcares = [], []

        for i in range(2 ** N):
            truth = [bool(int(s)) for s in list(format(i, '0%sb' % N))]
            rules_allowed = None
            for ii in range(len(hkl_list)): ########## use index and not hkl
                if truth == [truthList[ii] for truthList in ruleTruths]:  #truth pattern matches pattern in truth table
                    #if allowed(hkl_list[ii]) == rules_allowed or rules_allowed == None:
                    if allowed(hkl_list[ii], allowed_list) == rules_allowed or rules_allowed == None:
                        #rules_allowed = allowed(hkl_list[ii])
                        rules_allowed = allowed(hkl_list[ii], allowed_list)

                    else:
                        raise Exception('inconsistent rules:', truth, ' same set of truths have reflections that are allowed and forbidden so no combination of the rules can work') # 

            if rules_allowed == None:
                dontcares += [truth]
            elif rules_allowed == True:
                minterms += [truth]

        return (minterms, dontcares)

    N = len(truthListListByRuleSmall)
    for rules_combination in combos(N):
        rules_comb_0 = [i-1 for i in rules_combination] # subtract 1 so rule 1 is list item 0
        smallrules = [truthListListByRuleSmall[i] for i in rules_comb_0]
        #getMinterms1(reflist, smallrules)
        #print(rules_combination)
        try:
            (minterms, dontcares) = getMinterms1(reflist, smallrules)
            symList = symbols(['r%i' % i for i in rules_combination]) # create a list of symbols, one per rule
            compRule = POSform(symList, minterms, dontcares)
            display(compRule)
            #display(POSform(symList, minterms, dontcares))
            print('Found consistent combination of rules', rules_combination)
            print('====================', len(smallrules), len(minterms), len(dontcares))
            break # stop after the first consistent compound rule (remove break to compute all)
        except:
            compRule = None   #if can't find a compound rule
            #print('rules inconsistent')
    
    return compRule

    
for site in out[sgNum-1][1].keys():
    for class_no in out[sgNum-1][1][site]['classes'].keys():      
        print('\n\nSpacegroup #%i Site: %s Class: %i ranks: ' % (sgNum, site, class_no), out[sgNum-1][1][site]['classes'][class_no]['ranks'])
        findCompoundRule(out[sgNum-1][1][site]['classes'][class_no]['refs'])


        
# evaluate allRulesSmall for each ref in class

getTruthListfromReflist(out[sgNum-1][1][site]['classes'][class_no]['refs'], rulesDict)

        

In [ ]:
allRulesSmall

In [ ]:
#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
#allowed_list = out[sgNum-1][1]['c']['classes'][2]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
#allowed_list = out[sgNum-1][1]['e']['classes'][1]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
allowed_list = out[sgNum-1][1]['e']['classes'][5]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help




#should be no refs that are allowed but volate R1

# num_refs = 0
# print('hkl\t\tR1\tallowed')
# for ii in range(len(reflist)):
#     if allowed(reflist[ii], allowed_list): #### print only allowed
#     #if True:                                #### print all refs
#         print(reflist[ii], '\t', smallrules[0][ii],'\t', int(allowed(reflist[ii], allowed_list)))
#         num_refs += 1
#     if int(allowed(reflist[ii], allowed_list)) and not smallrules[0][ii]:
#         raise('something went wrong with rule R1')        
# print(num_refs, ' reflections')




num_refs = 0
print('hkl\t\tallowed\tValid rules')
for ii in range(len(reflist)):
    if allowed(reflist[ii], allowed_list): #### print only allowed
    #if True:                                #### print all refs
        tlist = []
        for jj in range(len(smallrules)):
            tlist += [smallrules[jj][ii]]
        #print(reflist[ii], '\t', int(allowed(reflist[ii], allowed_list)), '\t', smallrules[0][ii], )
        print(reflist[ii], '\t', int(allowed(reflist[ii], allowed_list)), '\t', tlist)
        num_refs += 1
    if int(allowed(reflist[ii], allowed_list)) and not smallrules[0][ii]:
        raise('something went wrong with rule R1')        
print(num_refs, ' reflections')


######## fix this: keep only Laue unique reflections
###############
##############


num_refs = 0
print('hkl\t\tallowed\tValid rules')
refs = []
for ii in range(len(reflist)):
    #check to see if any of the Laue equivalents of reflist[ii] is already in refs
    laue_equiv = laueClassEquivRefs(reflist[ii], laueSymList)
    if set(tuple(i) for i in refs) & set(tuple(i) for i in laue_equiv) == set():
        #laue_equiv.sort()
        laue_equiv.sort(key = lambda hh : hh[0] + 0.99 * hh[1] + 0.98 * hh[2], reverse = True)
        ref = laue_equiv[0]
        refs += [ref]
        if allowed(reflist[ii], allowed_list): #### print only allowed
            tlist = []
            for jj in range(len(smallrules)):
                tlist += [smallrules[jj][ii]]
            #print(reflist[ii], '\t', int(allowed(reflist[ii], allowed_list)), '\t', smallrules[0][ii], )
            print(ref, '\t', int(allowed(reflist[ii], allowed_list)), '\t', tlist)
            num_refs += 1
print(num_refs, ' reflections')






#### all seem to satisfy R1
#### next:
#### show truth of all rules (R2, R3, R4...) - should be no simple patern (all true or all false or some function of rules that is all true)
####     not quite right: simple pattern might not give a rule as there may be reflections that are forbidden but satisfty the rule.
#### ignore reflections that are related thought Laue symmetry
#### find new rules by eye
#### compare to set of rules - what are the new ones?

In [ ]:
laueSymList

In [ ]:
### test rules derived above - temp code - delete later

#example d1
#allowed_list = out[sgNum-1][1]['d']['classes'][1]['refs']
allowed_list = out[sgNum-1][1]['a']['classes'][2]['refs']

def allowed(hkl, allowed_list):
    return tuple(hkl) in allowed_list


import numpy as np

truthListList = []
for ref in tt.keys():
    #print(ref, list(map(int,tt[ref])))
    truthListList += [list(map(int,tt[ref]))]
truthListListByRuleBig = np.array(truthListList).T.tolist() # includes duplicates

#remove duplictaes rules/truths
truthListListByRuleSmall = []
for ruleNum in range(len(truthListListByRuleBig)):
    if truthListListByRuleBig[ruleNum] in truthListListByRuleSmall:
        print('Rule %i duplicated' % ruleNum)
    else:
        truthListListByRuleSmall += [truthListListByRuleBig[ruleNum]]

len(truthListListByRuleSmall)

#example: d1
#compound_rule = np.logical_and(truthListListByRuleSmall[1 - 1], np.logical_not(truthListListByRuleSmall[6 - 1])) # R1^¬R6
compound_rule = np.logical_and(np.logical_not(truthListListByRuleSmall[4 - 1]), np.logical_not(truthListListByRuleSmall[5 - 1])) # R1^¬R6


n_errors = 0
for ii in range(len(reflist)):
    #print(reflist[ii], compound_rule[ii], allowed(reflist[ii], allowed_list))
    if compound_rule[ii] != allowed(reflist[ii], allowed_list):
        print(reflist[ii], compound_rule[ii], allowed(reflist[ii], allowed_list))
        n_errors += 1
print('No. of errors: ', n_errors)

#print('no. minterms: ', len(minterms))
#print('no. dontcares: ',len(dontcares))

##########################################
# 
# seems to work now (using np.logical_and): test other cases
# 
##########################################

In [ ]:
rulesDict

In [ ]:
### delete


#print(len(tt[(0,0,0)]))
#print(len(rulesDict['nonTransCondRules']))
#print(rulesDict['transRule'])
#print(rulesDict['nonTransCondRules'][3])

#first 20 refs for selected classes
print('c1')
print(out[sgNum-1][1]['c']['classes'][1]['refs'][0:20])
print('c2')
print(out[sgNum-1][1]['c']['classes'][2]['refs'][0:20])
print('e1')
print(out[sgNum-1][1]['e']['classes'][1]['refs'][0:20])
print('e5')
print(out[sgNum-1][1]['e']['classes'][5]['refs'][0:20])


def showTruths(hkl):
    truths = getTruthListfromReflist([hkl], rulesDict)
    print('hkl = ', hkl)
    print('\n', rulesDict['transRule'], ':\t', truths[hkl][0])
    print()
    for i in range(len(rulesDict['nonTransCondRules'])):
        assert len(rulesDict['nonTransCondRules']) == len(truths[hkl]) - 1
        print(rulesDict['nonTransCondRules'][i], ':\t', truths[hkl][i+1])
        print()

showTruths((1, 0, 1))
print('\n\n')
showTruths((0, 0, 2))




In [ ]:
########## delete
display(allRulesSmall)
print(len(allRulesSmall))

print('Rules used:')
#for i in range(1, len(allRulesSmall)):
for i in range(0, len(allRulesSmall)):
    print('r%i:\t' % i, allRulesSmall[i])

########## rule 0 lost; lindex rules from 1?

print(combos(4))
print(rules_comb_0)

print('g1')
print(out[sgNum-1][1]['c']['classes'][1]['refs'])


## Label rules from 1 but index from 0
## Rules combos all to include rule 1 if primative but not therwise (currently only coded with one primative).
# Next:
## check Spacegroup #142 Site: g Class: 1 - why does is not require R1?

getTruthListfromReflist(out[sgNum-1][1][site]['classes'][class_no]['refs'], rulesDict)
- create small rulesDict (rules not duplicated) and run this command to check refs against rules.

## check: K=+1

(1,1,0)- allowed K=+1

(0,0,2) - forbidden K=+1

Look at tensor calc - why is this?


In [ ]:
# create rules dict with small set of rules for testing and test against reflections in class

# multiple copies for test - delete extra copies

site, cls = 'c', 1

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))


In [ ]:
#here123
# delete cell

#display(smallRulesDict)
#display(getTruthListfromReflist([(0, 1, 1)], smallRulesDict))
display(getTruthListfromReflist([(2, 2, 2)], smallRulesDict))
#4, 6, 10, 11 are all wrong

# hkl = (0, 1, 1)
# rule = 4
# #get first item from cond set
# for cond in smallRulesDict['nonTransCondRules'][rule -2][0]:
#     break


# display(cond)
# display(cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}))

# rule = smallRulesDict['nonTransCondRules'][rule -2][1]
# display(rule)
# sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0 # expression not even; rule broken
# # correct - True because rule is broken
# #so what is wrong??????

# ######## check indexing and extra rules at end ???


In [ ]:
# create rules dict with small set of rules for testing and test against reflections in class

site, cls = 'c', 2

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

In [ ]:
# create rules dict with small set of rules for testing and test against reflections in class

site, cls = 'e', 1

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

In [ ]:
# create rules dict with small set of rules for testing and test against reflections in class

site, cls = 'e', 5

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

In [ ]:
import cctbx